# webcam

In [1]:
import cv2
import numpy as np

# Abrir o vídeo
cam = cv2.VideoCapture("./teste4.mp4")

# Ler o primeiro frame como referência de fundo
ret, frame_reference = cam.read()
if not ret:
    print("Erro ao carregar o vídeo")
    cam.release()
    exit()

frame_reference = cv2.resize(frame_reference, (0, 0), fx=1, fy=1)
frame_reference = cv2.flip(frame_reference, 1)
frame_reference_gray = cv2.cvtColor(frame_reference, cv2.COLOR_BGR2GRAY)
frame_reference_gray = cv2.GaussianBlur(frame_reference_gray, (31, 31), 0)

# Inicializar variáveis para histórico de pessoas detectadas
people_detected = []
person_id = 0
TIMEOUT = 50  # Número de frames até considerarmos que a pessoa saiu
DETECTION_DELAY = 240  # Número de frames para considerar uma nova pessoa

# Função para calcular o histograma de cores de um ROI
def get_color_histogram(roi):
    hist = cv2.calcHist([roi], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    return cv2.normalize(hist, hist).flatten()

while True:
    ret, frame = cam.read()

    if not ret:
        break
    
    frame = cv2.flip(frame, 1)
    frame = cv2.resize(frame, (0, 0), fx=1, fy=1)
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame_gray = cv2.GaussianBlur(frame_gray, (31, 31), 0)

    # Subtrair o frame de referência do frame atual
    frame_delta = cv2.absdiff(frame_reference_gray, frame_gray)
    _, thresh = cv2.threshold(frame_delta, 25, 255, cv2.THRESH_BINARY)
    thresh = cv2.dilate(thresh, None, iterations=2)

    # Encontrar contornos
    contours, _ = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Filtrar contornos pequenos e criar uma lista de áreas
    valid_contours = []
    areas = [cv2.contourArea(contour) for contour in contours if cv2.contourArea(contour) >= 1000]

    if areas:
        max_area = max(areas)

        for contour in contours:
            area = cv2.contourArea(contour)
            if area >= 1000 and area > max_area * 0.3:
                valid_contours.append(contour)

    # Loop sobre os contornos filtrados
    for contour in valid_contours:
        x, y, w, h = cv2.boundingRect(contour)
        roi = frame[y:y+h, x:x+w]
        person_hist = get_color_histogram(roi)

        new_person = True
        for person in people_detected:
            dist = cv2.compareHist(person['hist'], person_hist, cv2.HISTCMP_CORREL)
            if dist > 0.8:
                person['last_seen'] = (x, y)
                person['missed_frames'] = 0
                person['detection_delay'] += 1  # Incrementa o contador de frames para essa pessoa
                new_person = False
                break

        if new_person:
            # Adiciona nova pessoa apenas após o número definido de frames
            if person_id < DETECTION_DELAY:
                person_id += 1
                people_detected.append({
                    'id': person_id,
                    'hist': person_hist,
                    'last_seen': (x, y),
                    'missed_frames': 0,
                    'detection_delay': 1  # Inicia o contador para nova detecção
                })
                cv2.putText(frame, f"Pessoa {person_id} detectada", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            else:
                # Se a nova pessoa ainda não tiver sido detectada por um tempo suficiente, apenas atualiza last_seen
                cv2.putText(frame, "Esperando por nova detecção...", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Desenhar a casca convexa ao redor da pessoa detectada
        hull = cv2.convexHull(contour)
        cv2.drawContours(frame, [hull], -1, (0, 255, 0), 2)

    # Atualiza o contador de frames perdidos para todas as pessoas
    for person in people_detected:
        if person['missed_frames'] > TIMEOUT:
            people_detected.remove(person)
        else:
            person['missed_frames'] += 1

    # Mostrar IDs de todas as pessoas detectadas recentemente
    for person in people_detected:
        if person['missed_frames'] <= TIMEOUT:
            cv2.putText(frame, f"Pessoa {person['id']}", person['last_seen'], cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Mostrar o frame com as detecções
    cv2.imshow('Deteccao de Mudancas', frame)
    cv2.imshow('Thresh', thresh)
    cv2.imshow('Frame Delta', frame_delta)

    # Pressione 'q' para sair
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

# Liberar o vídeo e fechar as janelas
cam.release()
cv2.destroyAllWindows()


# vídeo de teste

In [2]:
import cv2
import numpy as np
import time

# Abrir o vídeo
cam = cv2.VideoCapture("./teste11.mp4")

# Ler o primeiro frame como referência de fundo
ret, frame_reference = cam.read()
if not ret:
    print("Erro ao carregar o vídeo")
    cam.release()
    exit()

fast = cv2.FastFeatureDetector_create() 
orb = cv2.ORB_create()

frame_reference = cv2.resize(frame_reference, (0, 0), fx=1, fy=1)
frame_reference = cv2.flip(frame_reference, 1)
frame_reference_gray = cv2.cvtColor(frame_reference, cv2.COLOR_BGR2GRAY)
frame_reference_gray = cv2.GaussianBlur(frame_reference_gray, (31, 31), 0)

# Inicializar variáveis para histórico de pessoas detectadas
people_detected = []
person_id = 0
false_id = 500000
TIMEOUT = 25  # Número de frames até considerarmos que a pessoa saiu
DETECTION_DELAY = 100  # Número de frames para considerar uma nova pessoa
paused = False

# Função para calcular o histograma de cores de um ROI
def get_color_histogram(roi):
    hist = cv2.calcHist([roi], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    return cv2.normalize(hist, hist).flatten()


COLOR_NAMES = {
    'preto': (0, 0, 0),
    'branco': (255, 255, 255),
    'cinza escuro': (64, 64, 64),
    'cinza': (128, 128, 128),
    'cinza claro': (192, 192, 192),
    'vermelho': (255, 0, 0),
    'verde': (0, 255, 0),
    'azul': (0, 0, 255),
    'amarelo': (255, 255, 0),
    'ciano': (0, 255, 255),
    'magenta': (255, 0, 255),
    'laranja': (255, 165, 0),
    'rosa': (255, 192, 203),
    'roxo': (128, 0, 128),
    'marrom': (165, 42, 42),
    'azul claro': (173, 216, 230),
    'verde claro': (144, 238, 144),
    'vermelho claro': (255, 102, 102),
    'amarelo claro': (255, 255, 153),
    'laranja escuro': (255, 140, 0),
    'roxo claro': (216, 191, 216),
    'azul escuro': (0, 0, 139),
    'verde escuro': (0, 100, 0),
    'vermelho escuro': (139, 0, 0),
    'dourado': (255, 215, 0),
    'prata': (192, 192, 192),
    'champanhe': (247, 231, 206),
    'violeta': (238, 130, 238),
    'salmão': (250, 128, 114),
    'menta': (189, 252, 201),
    'lavanda': (230, 230, 250),
    'azul marinho': (0, 0, 128),
    'verde oliva': (107, 142, 35),
    'bege': (245, 245, 220),
    'turquesa': (64, 224, 208),
    'marfim': (255, 255, 240)
}

# Função para calcular a distância euclidiana entre cores
def euclidean_distance(color1, color2):
    return np.sqrt(np.sum((np.array(color1) - np.array(color2)) ** 2))

# Função para encontrar o nome da cor mais próxima
def get_color_name(mean_color):
    closest_color = None
    min_distance = float('inf')
    for color_name, color_value in COLOR_NAMES.items():
        dist = euclidean_distance(mean_color[:3], color_value)  # Ignorar o canal alpha (transparência)
        if dist < min_distance:
            min_distance = dist
            closest_color = color_name
    return closest_color


def get_median_color(roi_contour, mask):
    # Extraímos as cores do ROI usando a máscara
    roi_colors = roi_contour[mask == 255]  # Pega apenas os pixels dentro da máscara
    if roi_colors.size == 0:
        return (0, 0, 0)  # Retorna preto se não houver pixels válidos

    # Calcula a mediana para cada canal (B, G, R)
    median_color = np.median(roi_colors, axis=0)
    return median_color

while True:
    if not paused:
        ret, frame = cam.read()

        if not ret:
            break
        
        frame = cv2.flip(frame, 1)
        frame = cv2.resize(frame, (0, 0), fx=1, fy=1)
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frame_gray = cv2.GaussianBlur(frame_gray, (31, 31), 0)

        # Subtrair o frame de referência do frame atual
        frame_delta = cv2.absdiff(frame_reference_gray, frame_gray)
        _, thresh = cv2.threshold(frame_delta, 25, 255, cv2.THRESH_BINARY)
        thresh = cv2.dilate(thresh, None, iterations=2)
        
        colored_movement = cv2.bitwise_and(frame, frame, mask=thresh)

        # Encontrar contornos
        contours, _ = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Filtrar contornos pequenos e criar uma lista de áreas
        valid_contours = []
        areas = [cv2.contourArea(contour) for contour in contours if cv2.contourArea(contour) >= 1000]

        if areas:
            max_area = max(areas)

            for contour in contours:
                area = cv2.contourArea(contour)
                if area >= 1000 and area > max_area * 0.3:
                    valid_contours.append(contour)

        # Loop sobre os contornos filtrados
        for contour in valid_contours:
            x, y, w, h = cv2.boundingRect(contour)
            roi = frame[y:y+h, x:x+w]
            person_hist = get_color_histogram(roi)

            new_person = True
            for person in people_detected:
                dist = cv2.compareHist(person['hist'], person_hist, cv2.HISTCMP_CORREL)
                if dist > 0.8:
                    person['last_seen'] = (x, y)
                    person['missed_frames'] = 0
                    person['detection_delay'] += 1  # Incrementa o contador de frames para essa pessoa
                    new_person = False
                    
                    # Se a pessoa já estiver detectada há mais de 3 segundos, capturar keypoints e descritores
                    if time.time() - person['time_entered'] > 3:
                        if person['false_id'] != 0: 
                            person_id += 1
                            person["real_id"] = person_id
                        person['false_id'] = 0

                        hull = cv2.convexHull(contour)
                        cv2.drawContours(frame, [hull], -1, (0, 255, 0), 2)

                        kp = fast.detect(colored_movement, None) 
                        kp, des = orb.compute(colored_movement, kp) 
                        person['keypoints'] = kp[:person['num_descriptors']]
                        person['descriptors'] = des[:person['num_descriptors']]
                        person['num_descriptors'] += 10  # Aumenta o número de descritores capturados

                        # Desenhar keypoints na imagem
                        cv2.putText(frame, f"Pessoa {person['real_id']} detectada", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

                    # Adicionar o código para calcular a cor predominante dentro do contorno
                    roi_contour = frame[y:y+h, x:x+w]
                    mask = np.zeros_like(roi_contour)
                    
                    print(x, y, w, h)
                    
                    cv2.drawContours(mask, [contour], -1, (255, 255, 255), thickness=cv2.FILLED, offset=(-x, -y))
                    median_color = get_median_color(roi_contour, mask[:, :, 0])
                    median_color = [float(f"{median_color[0]:.1f}"), float(f"{median_color[1]:.1f}"), float(f"{median_color[2]:.1f}")]
                    color_name = get_color_name(median_color)
                    cv2.putText(frame, f"Cor: {median_color}", (x, y - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)
                    cv2.putText(frame, f"Cor: {color_name}", (x, y + 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)
                    
                    break

            if new_person:
                false_id += 1
                people_detected.append({
                    'false_id': false_id,
                    'real_id': false_id,
                    'hist': person_hist,
                    'last_seen': (x, y),
                    'missed_frames': 0,
                    'detection_delay': 1,  # Inicia o contador para nova detecção
                    'time_entered': time.time(),
                    'num_descriptors': 10,  # Inicia com 10 descritores
                    'keypoints': None,
                    'descriptors': None
                })


                # cv2.putText(frame, f"Pessoa {person_id} detectada", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            # Desenhar a casca convexa ao redor da pessoa detectada
            # hull = cv2.convexHull(contour)
            # cv2.drawContours(frame, [hull], -1, (0, 255, 0), 2)

        # Atualiza o contador de frames perdidos para todas as pessoas
        for person in people_detected:
            if person['missed_frames'] > TIMEOUT:
                people_detected.remove(person)
            else:
                person['missed_frames'] += 1

        # Mostrar IDs de todas as pessoas detectadas recentemente
        # for person in people_detected:
        #     if person['missed_frames'] <= TIMEOUT:
        #         cv2.putText(frame, f"Pessoa {person['false_id']}", person['last_seen'], cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Mostrar o frame com as detecções
        # cv2.imshow('Thresh', thresh)
        # cv2.imshow('Frame Delta', frame_delta)
        cv2.imshow('Colored', colored_movement)
        cv2.imshow('Deteccao de Mudancas', frame)

    key = cv2.waitKey(30)
    if key == 27:  # Esc key to exit
        break
    elif key == ord('p'):  # Pause
        paused = True
    elif key == ord('o'):  # Continue
        paused = False

# Liberar o vídeo e fechar as janelas
cam.release()
cv2.destroyAllWindows()


777 0 71 54
749 346 77 125
0 138 55 122
709 56 46 98
776 0 72 55
748 342 80 136
0 138 57 122
799 123 49 54
708 39 47 116
776 0 72 55
748 339 83 139
0 139 79 122
803 122 45 61
776 0 72 55
673 0 48 61
786 405 46 73
673 0 65 151
748 338 83 140
0 137 123 161
673 0 65 152
206 0 156 79
748 337 84 141
0 137 123 161
206 8 129 71
673 0 84 156
748 338 83 140
0 137 123 161
205 7 135 74
673 0 65 152
748 339 83 139
0 137 122 161
708 29 29 123
206 7 132 73
673 0 48 62
748 339 83 139
0 137 122 161
708 30 49 126
206 9 127 70
673 0 47 61
748 337 84 141
0 137 125 162
708 29 50 127
673 0 48 62
747 321 88 157
0 135 128 167
673 0 86 156
203 0 162 84
746 320 91 158
0 92 130 212
672 0 128 171
37 0 333 141
746 320 92 158
0 91 131 215
672 0 132 172
37 0 336 142
746 320 92 158
0 92 131 215
672 0 133 172
36 0 339 142
746 320 90 158
0 134 130 172
672 0 71 152
34 0 341 142
187 325 80 72
746 321 90 157
0 135 130 173
755 116 93 73
672 0 70 152
32 0 345 142
188 324 80 72
746 320 92 158
0 134 131 174
742 115 106 74
67